# Assignment 2 — Biologically Inspired Optimization, Adversarial Search, and Game-Theoretic Decision Making

COMP3410/6410 2025 S2

**Weight**: 30% (30 marks)  
**Due**: 11:55pm Friday Week 11  
**Submission**: **This single Jupyter Notebook** (runs end-to-end). 

**Unit Learning Outcomes (assessed):**
- **ULO1**: Describe the roles of various search techniques in AI and use appropriate tools to implement them.
- **ULO3**: Explain biologically inspired algorithms and their roles in AI, and implement such algorithms in different contexts including adversarial games. 

**How to use this notebook**
> 1. Fill in the markdown cells for your answers of theory, analysis, reflection.
> 2. Implement your solutions (GA, ACO, adversarial search, one-shot/repeated games) in code cells.
> 3. Ensure the notebook is fully reproducible: `Kernel → Restart & Run All` should complete without errors and regenerate your results.
> 4. Replace all **TODO(you)** placeholders with your own work.
> 5. Fix a random seed (and state it) for comparability of plots/tables.
> 6. Keep everything self-contained (no external files or internet calls); include any small helper data you need inside the notebook.
> 7. Show key plots/tables (convergence curves, payoff matrices, trajectories) inline and briefly explain what each shows.
> 8. Use clear headings and short captions so your logic is easy to follow.

**Submission checklist**

- [ ] All required sections completed (markdown + code).
- [ ] Seeds set and documented; results reproducible with Run-All.
- [ ] Plots/tables included and labeled.
- [ ] No errors on a clean run; outputs visible.
- [ ] Academic integrity: all sources (if any) cited; work is your own.

# Student Information
- **Name**: _TODO(you)_ 
- **Student ID**: _TODO(you)_ 
- **Email**: _TODO(you)_ 
---

# 0 Assignment Global Variables
## 0.1 Import Modules

In [ ]:
# Reproducibility & basic imports
# TODO(you) add any into this cell that you may need
import random, math, statistics, itertools, functools, time
from dataclasses import dataclass, field
from typing import List, Tuple, Dict, Any, Optional
import numpy as np

# plotting
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import List, Tuple, Dict
import math

If you get a ModuleNotFoundError (e.g., No module named 'matplotlib')
- Use the next cell to install the missing package(s) into the current environment (e.g., pip install matplotlib).
- After it finishes, comment out the install line(s) to avoid re-installing every run (e.g., # pip install matplotlib).
- Return to the above imports cell and run it again (or use Kernel → Restart & Run All to be safe).

In [ ]:
# pip install matplotlib

## 0.2 Reproducibility & Random Seeds — read me first

Your algorithms (GA, ACO, search sampling, etc.) use randomness. To make your results reproducible (so markers can rerun and get the same numbers/plots), we control the random generators with a seed.
- `GLOBAL_SEED` is the single source of truth for your notebook’s randomness.
- `random.seed(...)` and `np.random.seed(...)` initialise the Python and NumPy random number generators.
- `set_seed(seed)` is a helper you can call at the top of any experiment to lock in a specific seed and print it for the log.

**How you should use it (your TODO)**
- At the start of the notebook (or before an experiment block), call: `set_seed(42)  # TODO(you): choose a seed and keep it consistent for your main results`.
- When you run multiple experiments (e.g., sensitivity analysis), either:
  - keep the seed fixed to compare algorithmic changes fairly, or
  - vary seeds intentionally and report the list of seeds you used (e.g., 5 runs with seeds [1, 2, 3, 4, 5]) and show mean ± std.
  
**When to change the seed**
- Change the seed only when you **intend to sample a different random run**, e.g., to show robustness.
- For your **primary results** (tables/plots in the report), keep one seed so outputs are exactly reproducible.

In [ ]:
# Set a global seed for reproducibility (you may change this but keep it logged)
GLOBAL_SEED = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

def set_seed(seed:int=42):
    global GLOBAL_SEED
    GLOBAL_SEED = seed
    random.seed(seed)
    np.random.seed(seed)
    print(f"Seed set to {seed}")

# 1 Project Narrative

You work for a **drone delivery company** operating under battery and time-window constraints and competing with a rival fleet. This is a generalisation of the Traveling Salesman Problem (TSP), known as the **Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)**. Unlike TSP, it involves constraints and multiple routes.

A drone delivery company must schedule operations each day:
- A fleet of drones must deliver packages to multiple locations.
- Each drone has a battery capacity limit (maximum distance it can travel before recharging).
- Each location has a time window when delivery is allowed (e.g., between 09:00–12:00).

This project is completed in three parts inside one notebook: 
1. **Biologically Inspired Optimization (GA & ACO)** for CVRPTW.
2. **Adversarial Search** (Negamax Plus $\alpha–\beta$ pruning) in competitive routing.
3. **Game-Theoretic Analysis** (payoff matrices, equilibria, repeated games). 

## 1.1 Dataset

This assignment uses one fixed dataset for the entire project. **Do not modify any values.**

- Depot parameters: number of drones, per-drone battery budget (distance units), travel speed, operating day horizon.
- Requests: coordinates `(x, y)`, time window `[tw_start, tw_end]` in minutes (from 00:00), service time, and demand (use 1 unless extending capacity).

**How to use the dataset**: Use `INSTANCE` everywhere for data in all parts (GA/ACO, adversarial search engine, payoff construction). If you need distances, use `euclid((x1,y1),(x2,y2))`.

In [ ]:
# === OFFICIAL SINGLE DATASET (INLINE, DO NOT MODIFY) ==========================

@dataclass
class Request:
    id: int
    x: float
    y: float
    tw_start: float   # minutes from 00:00 (e.g., 540 = 09:00)
    tw_end: float
    service: float    # minutes
    demand: int       # use 1 if not modeling capacity beyond battery

@dataclass
class Instance:
    name: str
    depot: Tuple[float, float]
    drones: int
    battery: float
    speed: float
    day_start: float
    day_end: float
    requests: List[Request]

def build_instance(name: str, spec: Dict) -> Instance:
    d = spec["depot"]
    inst = Instance(
        name=name,
        depot=(float(d["x"]), float(d["y"])),
        drones=int(d["drones"]),
        battery=float(d["battery"]),
        speed=float(d["speed"]),
        day_start=float(d["day_start"]),
        day_end=float(d["day_end"]),
        requests=[
            Request(
                id=int(r["id"]),
                x=float(r["x"]), y=float(r["y"]),
                tw_start=float(r["tw_start"]), tw_end=float(r["tw_end"]),
                service=float(r["service"]), demand=int(r.get("demand", 1))
            ) for r in spec["requests"]
        ],
    )
    # Basic validation
    assert inst.drones > 0 and inst.battery > 0 and inst.speed > 0, f"{name}: invalid drones/battery/speed"
    assert inst.day_start < inst.day_end, f"{name}: invalid day horizon"
    ids = set()
    for q in inst.requests:
        assert q.id != 0, f"{name}: request id 0 reserved for depot"
        assert q.tw_start < q.tw_end, f"{name}: request {q.id} has invalid time window"
        assert q.id not in ids, f"{name}: duplicate request id {q.id}"
        ids.add(q.id)
    inst.requests.sort(key=lambda q: q.id)
    return inst

# Official single instance (20 requests)
OFFICIAL = {
    "depot": {
        "x": 50.0, "y": 50.0,
        "drones": 4, "battery": 100.0, "speed": 1.0,
        "day_start": 480.0,    # 08:00
        "day_end": 1140.0,     # 19:00
    },
    "requests": [
        # north-west cluster
        {"id": 1,  "x": 20.0, "y": 82.0, "tw_start": 540.0, "tw_end": 660.0, "service": 6.0, "demand": 1},
        {"id": 2,  "x": 28.0, "y": 76.0, "tw_start": 585.0, "tw_end": 720.0, "service": 6.0, "demand": 1},
        {"id": 3,  "x": 34.0, "y": 70.0, "tw_start": 600.0, "tw_end": 690.0, "service": 6.0, "demand": 1},
        {"id": 4,  "x": 18.0, "y": 65.0, "tw_start": 560.0, "tw_end": 660.0, "service": 6.0, "demand": 1},
        {"id": 5,  "x": 26.0, "y": 58.0, "tw_start": 630.0, "tw_end": 750.0, "service": 6.0, "demand": 1},

        # north-east cluster
        {"id": 6,  "x": 72.0, "y": 82.0, "tw_start": 570.0, "tw_end": 660.0, "service": 6.0, "demand": 1},
        {"id": 7,  "x": 80.0, "y": 76.0, "tw_start": 600.0, "tw_end": 720.0, "service": 6.0, "demand": 1},
        {"id": 8,  "x": 66.0, "y": 70.0, "tw_start": 585.0, "tw_end": 690.0, "service": 6.0, "demand": 1},
        {"id": 9,  "x": 78.0, "y": 64.0, "tw_start": 615.0, "tw_end": 735.0, "service": 6.0, "demand": 1},
        {"id": 10, "x": 70.0, "y": 60.0, "tw_start": 540.0, "tw_end": 630.0, "service": 6.0, "demand": 1},

        # south-east cluster
        {"id": 11, "x": 78.0, "y": 40.0, "tw_start": 660.0, "tw_end": 780.0, "service": 6.0, "demand": 1},
        {"id": 12, "x": 72.0, "y": 28.0, "tw_start": 600.0, "tw_end": 720.0, "service": 6.0, "demand": 1},
        {"id": 13, "x": 66.0, "y": 22.0, "tw_start": 585.0, "tw_end": 690.0, "service": 6.0, "demand": 1},
        {"id": 14, "x": 62.0, "y": 34.0, "tw_start": 540.0, "tw_end": 630.0, "service": 6.0, "demand": 1},
        {"id": 15, "x": 70.0, "y": 18.0, "tw_start": 690.0, "tw_end": 810.0, "service": 6.0, "demand": 1},

        # south-west cluster
        {"id": 16, "x": 30.0, "y": 24.0, "tw_start": 600.0, "tw_end": 720.0, "service": 6.0, "demand": 1},
        {"id": 17, "x": 24.0, "y": 18.0, "tw_start": 585.0, "tw_end": 690.0, "service": 6.0, "demand": 1},
        {"id": 18, "x": 36.0, "y": 16.0, "tw_start": 660.0, "tw_end": 780.0, "service": 6.0, "demand": 1},
        {"id": 19, "x": 40.0, "y": 32.0, "tw_start": 540.0, "tw_end": 630.0, "service": 6.0, "demand": 1},
        {"id": 20, "x": 22.0, "y": 30.0, "tw_start": 615.0, "tw_end": 735.0, "service": 6.0, "demand": 1},
    ],
}

INSTANCE = build_instance("Main", OFFICIAL)

# convenience helpers
def euclid(a: Tuple[float,float], b: Tuple[float,float]) -> float:
    return math.hypot(a[0]-b[0], a[1]-b[1])

print(f"Loaded INSTANCE '{INSTANCE.name}': "
      f"requests={len(INSTANCE.requests)}, drones={INSTANCE.drones}, battery={INSTANCE.battery}, "
      f"horizon=({INSTANCE.day_start},{INSTANCE.day_end})")
assert len(INSTANCE.requests) == 20, "INSTANCE must have 20 requests"

## 1.2 Metrics & Payoff (read first, then implement)

You’ll define a single source of truth for evaluation so the same logic is reused across GA/ACO, adversarial search, game-theory payoff tables.

**What you must measure (routing metrics)**
- **Total distance**: Sum of Euclidean travel for each drone route (including depot → first and last → depot).
- **Energy cost**: Distance × `ENERGY_PER_UNIT` (constant you define).
- On-time services vs late services:
    - Each served request has an **arrival time** (minutes from day start).
    - A service is **on time** if `arrival_time ≤ tw_end`. It may wait if arriving before `tw_start` (waiting is allowed and costs time, not distance).
    - A service is **late** if `arrival_time > tw_end` (discouraged; see penalties).
- **Unserved requests**: Requests that never appear in any drone’s route.
- Constraint violations (if you treat constraints as “hard”):
    - **Battery**: Total route distance per drone must be `≤ INSTANCE.battery`.
    - **Day horizon**: A drone must finish before `INSTANCE.day_end` (and not depart before `INSTANCE.day_start`).
    - **Time windows**: Decide your policy:
        - _Hard_: disallow late services; routes causing lateness are invalid.
        - _Soft_: allow lateness but penalise it (preferred for graded comparability).
- Makespan / total time (optional, but useful for analysis): The latest finish time across drones.

- **Payoff (used for game analysis)**: `revenue−energy_cost−late_penalties−unserved_penalties−violation_penalties (if any)`. 
    - **Revenue**: e.g., `REVENUE_PER_SERVICE × (# of served requests)` (Optionally weight by zones or time-of-day, but keep it linear.)
    - **Energy cost**: `ENERGY_PER_UNIT × total_distance`
    - **Late penalties** (soft windows): e.g., `LATE_PENALTY_PER_MIN × total_late_minutes`
    - **Unserved penalties**: e.g., `UNSERVED_PENALTY_PER_REQ × (# unserved)`
    - **Violation penalties** (if you permit “invalid” routes to return a finite score): very large constants to dominate the objective, or simply reject invalid routes and return `-inf`.
    
Edge-cases you must handle:
- Empty route for a drone (allowed).
- Multiple drones may visit the same request only if you count it once; simplest is: disallow duplicates across all routes.
- If you treat time windows as hard: a single late service invalidates the route (or entire solution) — be explicit.

Tip: Keep your definitions consistent across all parts. Keep the payoff linear/transparent. Avoid nonlinear squashes (e.g., tanh) because they break expectation properties you’ll rely on in adversarial search, game theory.

TODO(you) 

Reporting expectations (what to show in your notebook)

- A short table for your chosen constants.
- One example evaluation call showing the full metrics dict for a simple 2–3 route solution.
- A 2–3 sentence justification for soft vs hard time-window choice and battery policy.
- Confirm that the same evaluator is used in Part 1 (GA/ACO fitness), Part 2 (adversarial search), and Part 3 (payoff matrices).

In [ ]:
# code TODO(you) to implement Metrics & Payoff
# help you to start...
# tune once, reuse everywhere
REVENUE_PER_SERVICE      = 10.0
ENERGY_PER_UNIT          = 1.0        # cost per distance unit
LATE_PENALTY_PER_MIN     = 0.5        # soft window penalty
UNSERVED_PENALTY_PER_REQ = 15.0
VIOLATION_PENALTY        = 10_000.0   # if you decide to score invalid schedules
# You can adjust magnitudes 
# (e.g., make unserved more costly than late), 
# but report your values in your markdown and keep them constant for all experiments.

# function may include: 
# route_distance
# simulate_route_timing
# evaluate_solution
# etc.

# 2 Part 1-Biologically Inspired Optimisation (GA & ACO) 

You must design, implement, and evaluate biologically inspired algorithms from Genetic Algorithms (GA) and Ant Colony Optimization (ACO) to solve the CVRPTW scheduling problem.

**Theoretical Design (Critical Thinking)** (Report)
1. Explain how this problem differs from the classical TSP, and why standard binary encoding is unsuitable.
2. Propose how you will adapt GA to solve this problem:
    - Candidate representation (permutation or alternative).
    - Crossover and mutation operators.
    - Fitness function design (must account for distance + battery + time windows).
3. Propose how you will adapt ACO to solve this problem:
    - Pheromone representation.
    - Visibility ($\eta$) and how it incorporates both distance and time feasibility.
    - Evaporation rules.
4. Critically discuss trade-offs between GA and ACO for this problem (exploration, exploitation, scalability).

**Implementation (Python Project)** (Code)
1. Implement Genetic Algorithm (GA):
    - Representation: routes assigned to drones (permutation or multi-route encoding).
    - Variation: ordered crossover, plus mutation adapted for multi-route constraints.
    - Fitness: penalise infeasible solutions (exceeding battery/time windows).
2. Implement Ant Colony Optimization (ACO):
    - Transition probability formula based on pheromone + visibility.
    - Pheromone update with evaporation and deposit.
    - Ensure feasible route construction under constraints.
3. Apply your implementations on provided dataset and with pre-defined metrics.

**Experimental Study (Analysis)** (Code + Report)
1. Report best and average performance for each algorithm on the dataset.
2. Present convergence behaviour (fitness vs generation/iteration).
3. Perform parameter sensitivity analysis:
    - For GA: vary population size or mutation rate.
    - For ACO: vary $\alpha$, $\beta$, and evaporation rate $\rho$.
4. Discuss how design choices affected performance, linking results to your theoretical expectations.

**Reflection (Critical Evaluation)** (Report)
1. Which algorithm performed better, and why?
2. Did your theoretical expectations (Part 1) match experimental results? Why or why not?
3. Propose one novel extension (e.g., hybrid GA–ACO, adaptive mutation/evaporation) and explain how it could improve performance for large-scale problems (>1000 locations).
4. Reflect on the strengths and weaknesses of your implementations.

_Report length: approximately 800 words (total for 2.1, 2.2, 2.3 and 2.4). There is no length limit for code._

## 2.1 Theoretical Design

TODO(you)

## 2.2 Implementation (Python Project)

In [ ]:
# TODO(you)

## 2.3 Experimental Study

In [ ]:
# TODO(you) Code can be combined in 2.2. Report must be written under 2.3

TODO(you) analysis for Experimental Study

## 2.4 Reflection

TODO(you)

---
# 3 Part 2-Adversarial Search (Negamax/$\alpha$–$\beta$) in Competitive Routing

Model the competition between two fleets as a turn-based, zero-sum game and implement Negamax with $\alpha–\beta$ pruning to choose actions. You must reuse the same evaluator from §1.2 to derive utilities from completed plans.

What you must submit in this part

**Theoretical Design (Critical Thinking)** (Report)

- Game model: define state, actions, transition, terminal condition.
- Utility: explain how you map the shared instance to two route sets and compute a zero-sum utility using your §1.2 evaluator.
- Pruning & ordering: describe your $\alpha–\beta$ implementation and any move ordering (e.g., greedy top-k by time-window urgency).
- Abstractions/limits: justify any depth limits, candidate pruning, or heuristic cutoffs to control branching.

**Implementation (Python Project)** (Code)

- A game engine with `State`, `legal_moves`, `apply_move`, `is_terminal`, `utility`.
- Negamax $\alpha–\beta$ with depth limit and (at least basic) move ordering.
- A demo that prints best move/value from the initial state and the number of nodes expanded.

**Reflection (Critical Evaluation)** (Report)

- What move does Negamax prefer from the start?
- Does move ordering reduce node expansions? (Show counts for “with vs without ordering” at fixed depth.)

_Report length for Part 2 theory: approximately 500–700 words, Reflection: 2–4 sentences. There is no length limit for code._

**Constraints**

- Use only the provided `INSTANCE`.
- Keep the game deterministic (no dice rolls here).
- Your utility must be consistent with §1.2 (same constants/policies).
- Respect computational limits: depth-limit and/or candidate pruning is encouraged; document your choices.

## 3.1 Theoretical Design

TODO(you)

## 3.2 Implementation (Python Project)

In [ ]:
# TODO(you)

TODO(you) analysis for Experimental Study (various settings in Implementation)

## 3.3 Reflection

TODO(you)

---
# 4 Part 3-Strategic Decision Making in Competitive Drone Routing

Your company has now faced repeated interactions with its competitor in the delivery market. Instead of a one-shot adversarial search, the competition is ongoing. Each day both fleets decide on strategies not just to maximise immediate payoff, but to anticipate the other’s moves, coordinate when beneficial, and adapt across repeated encounters. Market regulators may also introduce auction-based bidding for access to certain delivery slots (prime time windows, high-demand areas).

You must analyse this evolving competition using **game theory** for this part.

**Theoretical Analysis (Critical Thinking)** (Report)
1. Formal game model: players, strategy sets, outcome mapping, and payoffs (computed with your §1.2 evaluator).
2. Dominance & equilibria: identify dominant strategies (if any), compute Nash equilibrium (NE) for a simplified 2×2 game, and discuss Pareto optimality.
3. Repeated games: explain how strategies like Always Cooperate (AC), Always Defect (AD), Tit-for-Tat (TFT) can sustain cooperation; when they fail. (Tit-for-Tat: Start by cooperating. Then copy your opponent’s last move—cooperate if they cooperated last round; defect if they defected.)
4. (Optional) Auctions: briefly reason about first-price vs Vickrey bidding and truthful vs shaded behavior.
    
**Implementation (Python Project)** (Code)
1. One-shot game: build a small 2×2 payoff matrix from two concrete policies per player; auto-check best responses and NE.
2. Repeated game: simulate multiple rounds with strategy profiles and plot cumulative payoffs.
3. (Optional) Auctions: first-price & Vickrey simulation with a few valuations; short comparison.

**Reflection (Critical Evaluation)** (Report)
1. Show your 2×2 matrix, report NE (and whether it’s Pareto-optimal).
2. For repeated games, discuss the payoffs.

_Report length for Part 3 theory: approximately 500–700 words (total). There is no length limit for code._

## 4.1 Theoretical Design

TODO(you)

## 4.2 Implementation (Python Project)

In [ ]:
# TODO(you) one-shot game model

In [ ]:
# TODO(you) repeated game

TODO(you) analysis for Experimental Study (various settings in Implementation)

## 4.3 Reflection

TODO(you)

---
## Marking Rubric (30 marks total)
| Criterion | Marks | Description |
|---|---:|---|
| **Theoretical Design & Modeling** | 12 | Clear formalisation of the CVRPTW setting (vs TSP); justified GA (representation, operators, repair) and ACO (pheromone, visibility, updates) designs; coherent game-theoretic modelling in Part 3 (players, strategies, outcome mapping); correct identification/analysis of best responses, NE, Pareto; all fitness/utility definitions explicitly tied to the single §1.2 evaluator. |
| **Implementation** | 9 | Correct, readable, and modular code for GA, ACO, and Negamax α–β (plus repeated-game simulator); solutions run end-to-end on the provided inline dataset; consistent reuse of the same evaluator across Parts 1–3; seeding for reproducibility; code choices trace back to the Theory (e.g., operators ↔ constraints, move ordering ↔ urgency). |
| **Experimental Study** | 3 | Sound experiments with meaningful settings (e.g., generations/ants/depth, candidate pruning); clear plots/tables (e.g., convergence curves, 2×2 payoff matrix, repeated-game trajectories); results interpreted with adequate depth; findings connected to theory. |
| **Reflection & Critical Evaluation** | 6 | Insightful discussion comparing theory vs. empirical results; trade-offs (exploration/exploitation, feasibility handling, pruning effects); limitations and threats to validity (seed sensitivity, hyperparameters, abstraction gaps); original suggestions (e.g., hybridisation, improved visibility, TFT). |